YOLOv8 pretrained Detect models, includes YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, and YOLOv8x, etc are pretrained on the COCO dataset.

# Setup

In [ ]:
!pip install keras_nlp
!pip install keras-cv
!pip install roboflow
!pip install supervision
!pip install yolov8
!pip install supervisely
!pip uninstall pydantic -y
!pip install pydantic==2.5.0
!pip install Roboflow

In [9]:
import os
from tqdm.notebook import tqdm
import keras_cv
import tensorflow as tf
from tensorflow import keras
import xml.etree.ElementTree as ET
from keras_cv import bounding_box
from keras_cv import visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn import metrics
import supervision as sv
from ultralytics import YOLO

import glob
from PIL import Image

# Load data

In [ ]:
# Link to Kaggle and Load Data
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/kaggle.json

# Copy the API command from Kaggle, should be the following format
# !kaggle datasets download -d crowdflower/twitter-airline-sentiment -p ./airline_tweets

!kaggle datasets download -d snehilsanyal/construction-site-safety-image-dataset-roboflow -p ./construction_site
!unzip /content/construction_site/construction-site-safety-image-dataset-roboflow.zip

Load data from Roboflow with YOLOv8 format: https://universe.roboflow.com/roboflow-universe-projects/construction-site-safety/dataset/30#

Details of data set can be found in report:
The data file downloaded has already been split into train, validation, and test set.

Number of classes: 10
Because this task is specifically for construction site, we labels 10 categories that are of interest: Class Map {0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask', 4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone', 7: 'Safety Vest', 8: 'machinery', 9: 'vehicle'}

Label Annotation: YoloV8 format

NOTE: NEED TO GENERATE AN API KEY ON ROBOFLOW AND PUT IT WHERE rf = Roboflow(api_key="YOUR PRIVATE KEY")

Citation: @misc{ construction-site-safety_dataset,
    title = { Construction Site Safety Dataset },
    type = { Open Source Dataset },
    author = { Roboflow Universe Projects },
    howpublished = { \url{ https://universe.roboflow.com/roboflow-universe-projects/construction-site-safety } },
    url = { https://universe.roboflow.com/roboflow-universe-projects/construction-site-safety },
    journal = { Roboflow Universe },
    publisher = { Roboflow },
    year = { 2023 },
    month = { feb }
}


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="#####")
############# Need to change it to your private API key for ROBO
project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
version = project.version(28)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.1.25, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Construction-Site-Safety-28 in yolov8:: 100%|██████████| 5614/5614 [00:00<00:00, 6800.97it/s]


In [ ]:
# !pip install PyYAML
import yaml

with open("/content/Construction-Site-Safety-28/data.yaml", "r") as f:
    data_config = yaml.safe_load(f)

data_config["train"] = "/content/Construction-Site-Safety-28/train/images"
data_config["val"] = "/content/Construction-Site-Safety-28/valid/images"

with open("/content/Construction-Site-Safety-28/data.yaml", "w") as f:
    yaml.dump(data_config, f)

data_yaml = "/content/Construction-Site-Safety-28/data.yaml"


# Fine tuning pretrained models

Fine tuning YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, and YOLOv8x

The most typical pretrained models for YOLOv8 object detection models are YOLOv8n, YOLOv8s, YOLOv8m, YOLOv8l, and YOLOv8x, but here we pick three to test on to see. The differ the number of Layers, width of the Network, resolution of Input Images, and etc. So in general, they differ in complexity in their architectures, with YOLOv8x being the most complex, and YOLOv8n being the most simple and fastest one.

imgsz=640: This sets the image size to 640 pixels. In object detection models like YOLO, the input image is often resized to a standard size, in this case, 640x640 pixels.

half=False: This is specifying the precision of the calculations. If set to True, it might use half-precision (16-bit) floating-point numbers to speed up model inference, sacrificing a bit of accuracy for speed. 'False' means it's likely using full precision (32-bit).

## Fine tune yolov8n

Plots will be saved to content/runs/detect/train file, can check there

In [ ]:
# build a new model from YAML
model_fine_n = YOLO('yolov8n.yaml')

# load a pretrained model
model_fine_n = YOLO('yolov8n.pt')

# build from YAML and transfer weights
model_fine_n = YOLO('yolov8n.yaml').load('yolov8n.pt')

# Train the model
results = model_fine_n.train(data= data_yaml, epochs= 40, imgsz=640, patience = 3, plots=True)


Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Construction-Site-Safety-28/data.yaml, epochs=40, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning /content/Construction-Site-Safety-28/train/labels.cache... 2605 images, 6 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/004720_jpg.rf.afc486560a4004c7cfd67910af31a29c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/construction-813-_jpg.rf.b085952261fd98f2e76b8065de149b5f.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Construction-Site-Safety-28/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      19/40      3.21G      1.019      1.032      1.225        344        640: 100%|██████████| 163/163 [00:15<00:00, 10.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  8.19it/s]

                   all        114        697      0.801      0.644      0.701      0.364
Stopping training early as no improvement observed in last 3 epochs. Best results observed at epoch 16, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



19 epochs completed in 0.088 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.3MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


                   all        114        697      0.803      0.628      0.701      0.387
               Hardhat        114         79      0.916      0.687      0.808      0.495
                  Mask        114         21          1      0.738      0.808      0.508
            NO-Hardhat        114         69      0.839      0.507      0.605      0.237
               NO-Mask        114         74      0.699      0.459      0.524      0.221
        NO-Safety Vest        114        106      0.681      0.491      0.615      0.316
                Person        114        166      0.804      0.669       0.77      0.415
           Safety Cone        114         44      0.835      0.841       0.85      0.464
           Safety Vest        114         41       0.86      0.747      0.805      0.456
             machinery        114         55      0.831      0.709      0.781       0.45
               vehicle        114         42      0.561      0.429      0.449      0.304
Speed: 0.1ms preproce

#### get the test set performance

In [ ]:
# Changed the file name in order to use the validation function to find out test accurac
# test changes to valid
# valid change to valid1
old_dir_path = '/content/Construction-Site-Safety-28/valid'
new_dir_path = '/content/Construction-Site-Safety-28/valid1'

# Rename the directory
os.rename(old_dir_path, new_dir_path)

old_dir_path_test = '/content/Construction-Site-Safety-28/test'
new_dir_path_test = '/content/Construction-Site-Safety-28/valid'

# Rename the directory
os.rename(old_dir_path_test, new_dir_path_test)

In [ ]:
model_fine_n.val(data='/content/Construction-Site-Safety-28/data.yaml')

Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/Construction-Site-Safety-28/valid/labels... 82 images, 8 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<00:00, 1374.98it/s]

val: New cache created: /content/Construction-Site-Safety-28/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:07<00:00,  1.20s/it]


                   all         82        760       0.78      0.602      0.661      0.349
               Hardhat         82        110      0.968      0.727      0.867      0.501
                  Mask         82         28      0.973      0.679      0.749      0.426
            NO-Hardhat         82         41      0.607      0.439      0.447      0.161
               NO-Mask         82         79      0.634      0.506      0.555      0.228
        NO-Safety Vest         82         90      0.786      0.613      0.643      0.299
                Person         82        174      0.787      0.701      0.747      0.414
           Safety Cone         82         92      0.758      0.359      0.434      0.172
           Safety Vest         82         61      0.856      0.779      0.859      0.485
             machinery         82         44      0.799      0.724      0.787      0.496
               vehicle         82         41      0.627      0.488      0.517      0.308
Speed: 0.2ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ab9660d5d80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [ ]:
# Change the path back
old_dir_path = '/content/Construction-Site-Safety-28/valid'
new_dir_path = '/content/Construction-Site-Safety-28/test'

# Rename the directory
os.rename(old_dir_path, new_dir_path)

old_dir_path_test = '/content/Construction-Site-Safety-28/valid1'
new_dir_path_test = '/content/Construction-Site-Safety-28/valid'

# Rename the directory
os.rename(old_dir_path_test, new_dir_path_test)
Input_path = "/content/Construction-Site-Safety-28/test/images/000005_jpg.rf.96e9379ccae638140c4a90fc4b700a2b.jpg"

In [ ]:

results = model_fine_n.predict(source=Input_path, save=True)


image 1/1 /content/Construction-Site-Safety-28/test/images/000005_jpg.rf.96e9379ccae638140c4a90fc4b700a2b.jpg: 640x640 2 Hardhats, 1 NO-Hardhat, 3 NO-Masks, 4 Persons, 14.9ms
Speed: 3.3ms preprocess, 14.9ms inference, 3.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/train44


#### get the image annotation by our model

In [ ]:
test_path = "/content/Construction-Site-Safety-28/test/images"
test_path_annotate = "/content/Construction-Site-Safety-28/test/labels"

results = model_fine_n.predict(source=test_path, save=True)


image 1/82 /content/Construction-Site-Safety-28/test/images/-4405-_png_jpg.rf.82b5c10b2acd1cfaa24259ada8e599fe.jpg: 640x640 1 Person, 1 machinery, 12.2ms
image 2/82 /content/Construction-Site-Safety-28/test/images/000005_jpg.rf.96e9379ccae638140c4a90fc4b700a2b.jpg: 640x640 1 Hardhat, 1 NO-Hardhat, 8.4ms
image 3/82 /content/Construction-Site-Safety-28/test/images/002551_jpg.rf.ce4b9f934161faa72c80dc6898d37b2d.jpg: 640x640 1 Hardhat, 3 Persons, 2 Safety Vests, 1 machinery, 8.6ms
image 4/82 /content/Construction-Site-Safety-28/test/images/003357_jpg.rf.9867f91e88089bb68dc95947d5116d14.jpg: 640x640 1 Person, 1 Safety Cone, 1 machinery, 8.4ms
image 5/82 /content/Construction-Site-Safety-28/test/images/004063_jpg.rf.1b7cdc4035bcb24ef69b8798b444053e.jpg: 640x640 5 Hardhats, 1 NO-Safety Vest, 6 Persons, 1 Safety Vest, 1 machinery, 8.3ms
image 6/82 /content/Construction-Site-Safety-28/test/images/004763_jpg.rf.46484e6ca73caeaa9de45822cf1085a9.jpg: 640x640 2 Hardhats, 1 NO-Hardhat, 3 NO-Masks, 

In [ ]:
# Define the current and new folder paths for later usage (to check plots for each model)
old_folder_path = '/content/runs/detect/train'
new_folder_path = '/content/runs/detect/train_yolov8n_finetune'

# Transfer learning

## YOLOv8n BACKBONE freeze

In [ ]:
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze
    for k, v in model.named_parameters():
        v.requires_grad = True  # train all layers
        if any(x in k for x in freeze):
            print(f'freezing {k}')
            v.requires_grad = False
    print(f"{num_freeze} layers are freezed.")

In [ ]:
yolov8n_transfer = YOLO('yolov8n.yaml').load('yolov8n.pt')
for layer in yolov8n_transfer.modules():
    print(layer)

for name, parameter in yolov8n_transfer.named_parameters():
  layer_number = int(name.split('.')[2])
  if layer_number <= 9:
    parameter.requires_grad = False
  print(name, layer_number, parameter.requires_grad)

Transferred 355/355 items from pretrained weights
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=

In [ ]:
yolov8n_transfer.add_callback("on_train_start", freeze_layer)

In [ ]:
trainable_params = 0
trainable_layers = 0

for name, parameter in yolov8n_transfer.named_parameters():
  # print(parameter.requires_grad)
  if parameter.requires_grad:
        # Counting the number of trainable parameters
        trainable_params += parameter.numel()
        # Each layer with at least one trainable parameter is considered a trainable layer
        trainable_layers += 1


In [ ]:
trainable_params

1884528

In [ ]:
transfer_model = yolov8n_transfer.train(data= data_yaml, epochs= 40, imgsz=640, patience = 3, plots=True)

Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/Construction-Site-Safety-28/data.yaml, epochs=40, time=None, patience=3, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_

train: Scanning /content/Construction-Site-Safety-28/train/labels.cache... 2605 images, 6 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/004720_jpg.rf.afc486560a4004c7cfd67910af31a29c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/construction-813-_jpg.rf.b085952261fd98f2e76b8065de149b5f.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Construction-Site-Safety-28/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


Plotting labels to runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Freezing 10 layers
freezing model.0.conv.weight
freezing model.0.bn.weight
freezing model.0.bn.bias
freezing model.1.conv.weight
freezing model.1.bn.weight
freezing model.1.bn.bias
freezing model.2.cv1.conv.weight
freezing model.2.cv1.bn.weight
freezing model.2.cv1.bn.bias
freezing model.2.cv2.conv.weight
freezing model.2.cv2.bn.weight
freezing model.2.cv2.bn.bias
freezing model.2.m.0.cv1.conv.weight
freezing model.2.m.0.cv1.bn.weight
freezing model.2.m.0.cv1.bn.bias
freezing model.2.m.0.cv2.conv.weight
freezing model.2.m.0.cv2.bn.weight
freezing model.2.m.0.cv2.bn.bias
freezing model.3.conv.weight
freezing model.3.bn.weight
freezing model.3.bn.bias
freezing

...

      21/40      2.08G      1.103      1.255      1.273        352        640: 100%|██████████| 163/163 [00:13<00:00, 12.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  7.30it/s]

                   all        114        697      0.778      0.551      0.634      0.323
Stopping training early as no improvement observed in last 3 epochs. Best results observed at epoch 18, best model saved as best.pt.
To update EarlyStopping(patience=3) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



21 epochs completed in 0.113 hours.
Optimizer stripped from runs/detect/train5/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train5/weights/best.pt, 6.3MB

Validating runs/detect/train5/weights/best.pt...
Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
YOLOv8n summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.01s/it]


                   all        114        697      0.793      0.563      0.645      0.338
               Hardhat        114         79      0.934      0.658      0.747      0.446
                  Mask        114         21      0.739      0.762       0.77      0.489
            NO-Hardhat        114         69      0.746      0.406      0.537      0.209
               NO-Mask        114         74      0.761      0.405      0.473      0.195
        NO-Safety Vest        114        106      0.888      0.472      0.591      0.283
                Person        114        166      0.835      0.608      0.712      0.368
           Safety Cone        114         44      0.795       0.75      0.775      0.369
           Safety Vest        114         41      0.883      0.555      0.713      0.397
             machinery        114         55      0.724      0.572      0.677      0.394
               vehicle        114         42      0.629      0.444      0.455      0.232
Speed: 0.1ms preproce

In [ ]:
model_transfer_n = YOLO('/content/runs/detect/train5/weights/best.pt')

#### get the image annotation by our model

In [ ]:
test_path = "/content/Construction-Site-Safety-28/test/images"
results = model_transfer_n.predict(source=test_path, save=True)


image 1/82 /content/Construction-Site-Safety-28/test/images/-4405-_png_jpg.rf.82b5c10b2acd1cfaa24259ada8e599fe.jpg: 640x640 (no detections), 9.3ms
image 2/82 /content/Construction-Site-Safety-28/test/images/000005_jpg.rf.96e9379ccae638140c4a90fc4b700a2b.jpg: 640x640 2 Hardhats, 2 NO-Masks, 1 Person, 1 vehicle, 8.5ms
image 3/82 /content/Construction-Site-Safety-28/test/images/002551_jpg.rf.ce4b9f934161faa72c80dc6898d37b2d.jpg: 640x640 1 Hardhat, 1 NO-Mask, 3 NO-Safety Vests, 5 Persons, 8.9ms
image 4/82 /content/Construction-Site-Safety-28/test/images/003357_jpg.rf.9867f91e88089bb68dc95947d5116d14.jpg: 640x640 1 NO-Safety Vest, 1 Person, 1 Safety Cone, 8.4ms
image 5/82 /content/Construction-Site-Safety-28/test/images/004063_jpg.rf.1b7cdc4035bcb24ef69b8798b444053e.jpg: 640x640 4 Hardhats, 1 NO-Hardhat, 5 NO-Safety Vests, 7 Persons, 1 Safety Vest, 8.5ms
image 6/82 /content/Construction-Site-Safety-28/test/images/004763_jpg.rf.46484e6ca73caeaa9de45822cf1085a9.jpg: 640x640 3 Hardhats, 3 NO-

#### get the test set performance

In [ ]:
# Changed the file name in order to use the validation function to find out test accurac
# test changes to valid
# valid change to valid1
old_dir_path = '/content/Construction-Site-Safety-28/valid'
new_dir_path = '/content/Construction-Site-Safety-28/valid1'

# Rename the directory
os.rename(old_dir_path, new_dir_path)

old_dir_path_test = '/content/Construction-Site-Safety-28/test'
new_dir_path_test = '/content/Construction-Site-Safety-28/valid'

# Rename the directory
os.rename(old_dir_path_test, new_dir_path_test)

In [ ]:
model_transfer_n.val(data='/content/Construction-Site-Safety-28/data.yaml')

Ultralytics YOLOv8.1.25 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)


val: Scanning /content/Construction-Site-Safety-28/valid/labels.cache... 82 images, 8 backgrounds, 0 corrupt: 100%|██████████| 82/82 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  33%|███▎      | 2/6 [00:00<00:00,  4.01it/s]

WARNING ⚠️ NMS time limit 2.800s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:06<00:00,  1.16s/it]


                   all         82        760      0.753      0.557      0.616      0.303
               Hardhat         82        110      0.939      0.705      0.831      0.479
                  Mask         82         28      0.895       0.61      0.733      0.357
            NO-Hardhat         82         41      0.608      0.463      0.449      0.183
               NO-Mask         82         79      0.702      0.544      0.541      0.183
        NO-Safety Vest         82         90      0.786      0.456      0.584      0.265
                Person         82        174       0.77      0.672      0.704      0.343
           Safety Cone         82         92      0.683       0.38      0.392      0.151
           Safety Vest         82         61      0.743      0.656      0.757      0.403
             machinery         82         44      0.738       0.64       0.66      0.396
               vehicle         82         41      0.668      0.441      0.505      0.274
Speed: 0.2ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ab95ebec3a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

# Interface

Here, you can upload a random website picture to google drive

In [ ]:
# can be used if want to upload your own image and use the model to do object detection
# ! pip install -q kaggle
# from google.colab import files
# files.upload()

Get the bounding box for this image

In [ ]:
# test_path = ### Change to google drive path

# results = model_fine_n.predict(source=test_path, save=True)